In [1]:
from datetime import date, timedelta
import holidays
import pandas as pd

In [4]:
def calendar(year,country='ES',subdiv=None):
    """
    returns a dataframe with three columns:
    dates: with format [YYYY-MM-DD], ascending
    weekend: with value 1 if it's Saturday or Sunday, otherwise it returns 0.
    holidays: with value 1 if it's public holidays, otherwise it returns 0.
    """
    calendar=pd.DataFrame()

    def daterange(year):
        calendar = pd.DataFrame()
        calendar['dates']=pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D')
        return calendar

    calendario = daterange(year)

    def weekends(calendar):
        calendar['weekday']= calendar['dates'].apply(lambda x: x.weekday())
        weekend = {0:0,1:0,2:0,3:0,4:0,5:1,6:1}
        calendar['weekend'] = calendar['weekday'].map(weekend)
        calendar.drop(columns='weekday',inplace=True)
        return calendar

    calendario = weekends(calendario)

    def public_holidays(calendar):
        es_cl_holidays = holidays.country_holidays(country,subdiv)
        calendar['holidays']=calendar['dates'].apply(lambda x: x in es_cl_holidays).map(int)
        return calendar
    calendario = public_holidays(calendario)

    def times(calendario):
        times=['Morning','Evening','Night']
        for i in times:
            calendario[i] = pd.Series(0)
        return calendario
    calendario = times(calendario)
    return calendario



In [5]:
calendario = calendar(2022,'ES','CL')
calendario

,dates,weekend,holidays,Morning,Evening,Night
0,2022-01-01,1,1,0.0,0.0,0.0
1,2022-01-02,1,0,NaN,NaN,NaN
2,2022-01-03,0,0,NaN,NaN,NaN
3,2022-01-04,0,0,NaN,NaN,NaN
4,2022-01-05,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...
360,2022-12-27,0,0,NaN,NaN,NaN
361,2022-12-28,0,0,NaN,NaN,NaN
362,2022-12-29,0,0,NaN,NaN,NaN
363,2022-12-30,0,0,NaN,NaN,NaN


In [6]:
calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dates     365 non-null    datetime64[ns]
 1   weekend   365 non-null    int64         
 2   holidays  365 non-null    int64         
 3   Morning   1 non-null      float64       
 4   Evening   1 non-null      float64       
 5   Night     1 non-null      float64       
dtypes: datetime64[ns](1), float64(3), int64(2)
memory usage: 17.2 KB


In [15]:
def number_workers():
    worker= input("how many workers do you have? >>")
    try:
        worker = int(worker)
        return worker
    except ValueError:
        print("That's not a number!")

# Employees Dataframe

def df_workers(number_of_workers):

    def name_workers():
        name = input('How is your worker called >>')
        if name.strip().isdigit():
            print("That's not a name!")
        else:
            name=name.capitalize()
            return name

    def age_workers():
        age= input("How old is your worker? >>")
        try:
            age = int(age)
            return age
        except ValueError:
            print("That's not a number!")

    def salary_workers():
        salary = input("What's the annual salary of your worker? (In €) >>")
        try:
            salary = float(salary)
            return salary
        except ValueError:
            print("That's not a number!")

    def times_workers():
        timer =[1,2,3,4,5,6]
        time = input("When does your worker works? >>\n"
                "1 for Morning\n"
                "2 for Evening\n"
                "3 for Night\n"
                "4 for Morning and Evening\n"
                "5 for Morning and Night\n"
                "6 for Evening and Night\n")
        try:
            time = int(time)
            if time in timer:
                return time
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def gender_workers():
        genders ={1:'Female',2:'Male',3:'Non-binary'}
        gender = input("What's the gender of your worker? >>\n"
                    "1 for Female\n"
                    "2 for Male\n"
                    "3 for Non-binary\n")
        try:
            gender = int(gender)
            if gender in genders:
                return genders[gender]
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def weekend_workers():
        weekends ={1:'Yes',2:'No'}
        weekend = input("Does your worker work during weekends? >>\n"
                    "1 for No\n"
                    "2 for Yes\n")
        try:
            weekend = int(weekend)
            if weekend in weekends:
                return weekend-1
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def holiday_workers():
        holidays ={1:'Yes',2:'No'}
        holiday = input("Does your worker work during public holidays? >>\n"
                    "1 for No\n"
                    "2 for Yes\n")
        try:
            holiday = int(holiday)
            if holiday in holidays:
                return holiday - 1
            else: print('Select a correct value')
        except ValueError:
            print("That's not a number!")

    def data_workers(number_of_workers):
        data={}
        for i in range(number_of_workers):
            name = name_workers()
            age = age_workers()
            salary = salary_workers()
            schedule = times_workers()
            gender = gender_workers()
            weekend = weekend_workers()
            holiday = holiday_workers()
            data[i] = {'name': name,'age': age,'salary':salary,'schedule':schedule,'gender':gender,'weekend':weekend,'holiday':holiday}
        return data
    df = data_workers(number_of_workers)
    df = pd.DataFrame.from_dict(df,orient='index')
    return df


In [16]:
number_of_workers= number_workers()
number_of_workers
df_workers(number_of_workers)

,name,age,salary,schedule,gender,weekend,holiday
0,Pablo,23,23000.0,1,Male,1,1
1,Paula,49,30000.0,3,Female,0,0
2,Laura,62,32000.0,1,Female,1,1
